# START

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
def single_violin(expr,gene):
    # Create a violin plot of expression values
    plt.figure(figsize=(8, 6))
    sns.violinplot(y=expr, color="skyblue", inner="box")
    # Calculate statistics
    mean_expression = expr.mean()
    median_expression = expr.median()


    # Add mean and median lines
    plt.axhline(mean_expression, color="red",  linewidth=1)
    plt.axhline(median_expression, color="green",  linewidth=1)
    plt.text(0, mean_expression, f"Mean: {mean_expression:.2f}", color="red", fontsize=8)
    plt.text(0, median_expression, f"Median: {median_expression:.2f}", color="green", fontsize=8)

    # Customize the plot
    plt.title(f"Distribution of Expression Values for {gene}")
    plt.ylabel("Expression Value")
    plt.xlabel("All Samples (ROIs)")
    plt.show()

# Load the spatial data

In [2]:
import pandas as pd
import numpy as np
import os
from LEAPdata import get_slide_id, get_leap_ids

In [3]:
OME_LEVEL = 2 # should be level 1 but haven't extracted at 20x yet
DOWNSAMPLE = 2**OME_LEVEL
segment='CD45+'

file_path = 'C:/Users/hooll/Dropbox/19 PostGrad/00 CB Lab/01 projects/06 MultiModal/data/LEAP'
input_path = 'E:/01-data/LEAP'

In [4]:

sample_metadata = pd.read_excel(os.path.join(file_path,'latest',"meta_data.xlsx"), index_col=0)
samples = pd.read_excel(os.path.join(file_path,"samples.xlsx"), index_col=0)

In [5]:
samples.head()

,Biobank_ID,Slide_ID,H&E,Response,ECT_Carbo
LEAP_ID,,,,,
LEAP001,19005858,1,YES,Non-Responder,NO
LEAP003,21002266,2,YES,Non-Responder,NO
LEAP005,22001707,3,YES,Non-Responder,NO
LEAP008,22001709,4,YES,Responder,YES
LEAP009,19005859,5,YES,Responder,NO


In [6]:
sample_metadata = sample_metadata.set_index('DCC_ID')
sample_metadata.head()

,Scan.Name,Panel,Roi,Segment,Aoi,Area,Tags,Nuclei,ROICoordinateX,ROICoordinateY,...,NegGeoMean,NegGeoSD,NegProbes,lib_size,countOfLowEprGene,percentOfLowEprGene,ruv_W1,ruv_W2,ruv_W3,ruv_W4
DCC_ID,,,,,,,,,,,,,,,,,,,,,
DSP-1001660008694-G-A02.dcc,"LEAP015, LEAP016",(v1.0) Human NGS Whole Transcriptome Atlas RNA,1.0,CD45+,CD45+-aoi-001,7186.20,"PanCK+, CD45+, CD45+",119,76238.40625,34222.16797,...,1.008594,1.080060,False,22680,0,0.000000,-0.023221,-0.007044,-0.017423,0.022888
DSP-1001660008694-G-A03.dcc,"LEAP015, LEAP016",(v1.0) Human NGS Whole Transcriptome Atlas RNA,1.0,PanCK+,PanCK+-aoi-001,12989.46,"PanCK+, CD45+, PanCK+",239,76238.40625,34222.16797,...,1.115328,1.329805,False,51774,13492,73.087757,0.008550,-0.022099,0.023774,0.020598
DSP-1001660008694-G-A05.dcc,"LEAP015, LEAP016",(v1.0) Human NGS Whole Transcriptome Atlas RNA,2.0,PanCK+,PanCK+-aoi-001,14576.49,"PanCK+, CD45+, PanCK+",246,75587.60938,31947.06250,...,1.405232,1.600153,False,77467,11795,63.894908,0.006329,-0.020188,0.015802,-0.001984
DSP-1001660008694-G-A06.dcc,"LEAP015, LEAP016",(v1.0) Human NGS Whole Transcriptome Atlas RNA,3.0,CD45+,CD45+-aoi-001,5066.89,"PanCK+, CD45+, CD45+",85,76252.50000,29973.66797,...,1.017262,1.114309,False,20049,0,0.000000,-0.035865,-0.006889,-0.027053,0.014727
DSP-1001660008694-G-A07.dcc,"LEAP015, LEAP016",(v1.0) Human NGS Whole Transcriptome Atlas RNA,3.0,PanCK+,PanCK+-aoi-001,41874.55,"PanCK+, CD45+, PanCK+",796,76252.50000,29973.66797,...,3.029957,1.887786,False,274462,12122,65.666306,0.007790,-0.016314,0.018326,0.004932


In [7]:
sample_metadata.columns

Index(['Scan.Name', 'Panel', 'Roi', 'Segment', 'Aoi', 'Area', 'Tags', 'Nuclei',
       'ROICoordinateX', 'ROICoordinateY', 'Scan.Date', 'Scan.Width',
       'Scan.Height', 'Scan.Offset.X', 'Scan.Offset.Y', 'WTA.Atlas',
       'X.v1.0..Human.NGS.TCR.Profiling.RNA.Add.On', 'Sample_ID', 'nuclei',
       'area', 'NTC', 'NucleiZero', 'NucleiArea', 'AreaReads',
       'NegGeoMean_Hs_R_NGS_WTA_v1.0', 'NegGeoSD_Hs_R_NGS_WTA_v1.0', 'LOQ',
       'GenesDetected', 'GeneDetectionRate', 'Tissue_ID', 'LEAP_ID',
       'Patient_ID', 'Sample_Type', 'Biobank_ID', 'Response', 'Date_Sectioned',
       'Format', 'FORCE', 'ECT_Carbo', 'NACT_Group', 'sTILs', 'sTILs_Status',
       'BRCA1_Status', 'BRCA2_Status', 'Age', 'Ethnicity', 'Year_Diagnosis',
       'DistRecTime', 'DthTime', 'DistRecTime_Cat', 'DthTime_Cat', 'N_Stage',
       'T_Stage', 'Grade', 'RCB_Scores', 'RCB_Group', 'Extreme_NR',
       'Epithelial_E', 'Region_R', 'sample_id', 'NegGeoMean', 'NegGeoSD',
       'NegProbes', 'lib_size', 'countOfLo

In [8]:
#raw_matrix = pd.read_csv(os.path.join(file_path,"raw_exprs.csv"), index_col=0)
#expression_matrix = raw_matrix.astype(float)

In [9]:
#raw_matrix = pd.read_csv(os.path.join(file_path,'latest',"norm_counts.csv"), 
#                 sep=";",               # Semicolon separator
#                 decimal=",",          # Comma as decimal point
#                 quotechar='"',        # Handle quoted strings
#                 index_col=0)          # Use the first column as row index (gene name)

In [10]:
raw_matrix = pd.read_csv(os.path.join(file_path,'latest',"raw_counts.csv"), 
                 sep=";",               # Semicolon separator
                 decimal=",",          # Comma as decimal point
                 quotechar='"',        # Handle quoted strings
                 index_col=0)          # Use the first column as row index (gene name)

In [11]:

expression_matrix = raw_matrix.astype(float)

In [12]:
expression_matrix.head()

,DSP-1001660008694-G-A02.dcc,DSP-1001660008694-G-A03.dcc,DSP-1001660008694-G-A05.dcc,DSP-1001660008694-G-A06.dcc,DSP-1001660008694-G-A07.dcc,DSP-1001660008694-G-A08.dcc,DSP-1001660008694-G-A09.dcc,DSP-1001660008694-G-A10.dcc,DSP-1001660008694-G-A11.dcc,DSP-1001660008694-G-A12.dcc,...,DSP-1001660024022-D-H03.dcc,DSP-1001660024022-D-H04.dcc,DSP-1001660024022-D-H05.dcc,DSP-1001660024022-D-H06.dcc,DSP-1001660024022-D-H07.dcc,DSP-1001660024022-D-H08.dcc,DSP-1001660024022-D-H09.dcc,DSP-1001660024022-D-H10.dcc,DSP-1001660024022-D-H11.dcc,DSP-1001660024022-D-H12.dcc
A2M,5.0,11.0,12.0,2.0,26.0,12.0,25.0,1.0,53.0,7.0,...,21.0,12.0,314.0,20.0,7.0,52.0,134.0,19.0,59.0,458.0
ACADM,1.0,4.0,12.0,1.0,12.0,1.0,11.0,2.0,10.0,2.0,...,7.0,2.0,39.0,7.0,2.0,35.0,51.0,23.0,26.0,67.0
ACADS,1.0,2.0,3.0,1.0,6.0,1.0,8.0,1.0,6.0,2.0,...,4.0,7.0,28.0,5.0,4.0,20.0,22.0,5.0,5.0,44.0
ACAT1,1.0,9.0,7.0,1.0,45.0,1.0,11.0,1.0,9.0,4.0,...,7.0,9.0,55.0,2.0,5.0,35.0,57.0,15.0,34.0,47.0
ACVRL1,1.0,1.0,4.0,1.0,7.0,2.0,3.0,1.0,2.0,2.0,...,6.0,1.0,11.0,8.0,1.0,21.0,29.0,7.0,14.0,42.0


In [13]:
# Replace '.' with '-' in column names
expression_matrix.columns = expression_matrix.columns.str.replace('.', '-', regex=False)
expression_matrix.columns = expression_matrix.columns.str.replace('-dcc', '.dcc', regex=False)
expression_matrix.columns
# Ensure data is in a usable format
#print(expression_matrix.head())  # Check expression matrix
#expression_matrix.columns

Index(['DSP-1001660008694-G-A02.dcc', 'DSP-1001660008694-G-A03.dcc',
       'DSP-1001660008694-G-A05.dcc', 'DSP-1001660008694-G-A06.dcc',
       'DSP-1001660008694-G-A07.dcc', 'DSP-1001660008694-G-A08.dcc',
       'DSP-1001660008694-G-A09.dcc', 'DSP-1001660008694-G-A10.dcc',
       'DSP-1001660008694-G-A11.dcc', 'DSP-1001660008694-G-A12.dcc',
       ...
       'DSP-1001660024022-D-H03.dcc', 'DSP-1001660024022-D-H04.dcc',
       'DSP-1001660024022-D-H05.dcc', 'DSP-1001660024022-D-H06.dcc',
       'DSP-1001660024022-D-H07.dcc', 'DSP-1001660024022-D-H08.dcc',
       'DSP-1001660024022-D-H09.dcc', 'DSP-1001660024022-D-H10.dcc',
       'DSP-1001660024022-D-H11.dcc', 'DSP-1001660024022-D-H12.dcc'],
      dtype='object', length=5096)

## Only use data where we have both the gene expression and the metadata

keep only rows that are not pellets

In [14]:
sample_metadata = sample_metadata[sample_metadata['Sample_Type'] != 'PELLET']

In [15]:
rois_to_use = list(set(sample_metadata.index).intersection(set(expression_matrix.columns)))
len(rois_to_use)

4963

In [16]:
sample_metadata = sample_metadata.loc[rois_to_use]
expression_matrix = expression_matrix[rois_to_use]

In [17]:
sample_metadata.shape

(4963, 70)

# Identify hold out test set
need to group by patient


All segments from a patient must go into either train or test.

The test set must contain:

At least one CD45+ and one PanCK+ in Segment

At least one RESECTION and one DIAGNOSTIC in Sample_Type



In [18]:
patient_segment_counts = sample_metadata.groupby(['Patient_ID', 'Segment']).size().unstack(fill_value=0)
patient_segment_counts

Segment,CD45+,Full ROI,PanCK+,PanCK-,PanCK-/CD45-,TLS
Patient_ID,,,,,,
GST15102,2,0,2,0,3,0
GST17605,5,0,5,0,0,0
GST17723,7,0,7,0,5,0
GST18770,4,0,4,0,0,0
GST18928,5,0,5,0,0,0
...,...,...,...,...,...,...
GST48495,46,0,117,26,0,0
GST48581,23,0,21,0,0,0
GST48631,0,0,19,0,0,0


In [19]:
patient_segment_counts[(patient_segment_counts['CD45+'] == 0) & 
                                         (patient_segment_counts['PanCK+'] == 0)]

Segment,CD45+,Full ROI,PanCK+,PanCK-,PanCK-/CD45-,TLS
Patient_ID,,,,,,


In [20]:
patient_summary = sample_metadata.groupby('Patient_ID').agg({
    'Segment': lambda x: set(x),
    'Sample_Type': lambda x: set(x)
})
patient_summary

,Segment,Sample_Type
Patient_ID,,
GST15102,"{PanCK-/CD45-, CD45+, PanCK+}",{RESECTION}
GST17605,"{PanCK+, CD45+}",{RESECTION}
GST17723,"{PanCK-/CD45-, CD45+, PanCK+}",{RESECTION}
GST18770,"{PanCK+, CD45+}",{RESECTION}
GST18928,"{PanCK+, CD45+}",{RESECTION}
...,...,...
GST48495,"{PanCK+, PanCK-, CD45+}","{RESECTION, DIAGNOSTIC}"
GST48581,"{PanCK+, CD45+}",{DIAGNOSTIC}
GST48631,{PanCK+},"{RESECTION, DIAGNOSTIC}"


In [ ]:
# Step 1: Get unique Patient_IDs
unique_patients = sample_metadata['Patient_ID'].unique()

# Step 2: Randomly sample 10% of patients
np.random.seed(42)
sampled_patients = np.random.choice(unique_patients,
                                     size=int(0.1 * len(unique_patients)),
                                     replace=False)



In [41]:
# Step 3: Get all rows for those patients
train_metadata = sample_metadata[ ~sample_metadata['Patient_ID'].isin(sampled_patients)] #note this is the negation of isin
test_metadata = sample_metadata[sample_metadata['Patient_ID'].isin(sampled_patients)]

check we have totally separated test and train

In [46]:
(train_metadata.shape[0] + test_metadata.shape[0]) == sample_metadata.shape[0]

True

In [49]:
rois_train = list(set(train_metadata.index))
rois_test = list(set(test_metadata.index))

In [54]:
(len(rois_test)+len(rois_train))==len(rois_to_use)

True

In [56]:
expression_train = expression_matrix[rois_train]
expression_test = expression_matrix[rois_test]

In [55]:
print("Original Sample_Type proportions:")
print(sample_metadata['Sample_Type'].value_counts(normalize=True))

print("\nSampled Sample_Type proportions:")
print(train_metadata['Sample_Type'].value_counts(normalize=True))
print("\nSampled Sample_Type proportions:")
print(test_metadata['Sample_Type'].value_counts(normalize=True))

Original Sample_Type proportions:
Sample_Type
DIAGNOSTIC    0.812009
RESECTION     0.158775
Pre-NACT      0.015515
1_1           0.010478
2_2           0.003224
Name: proportion, dtype: float64

Sampled Sample_Type proportions:
Sample_Type
DIAGNOSTIC    0.821285
RESECTION     0.146363
Pre-NACT      0.017180
1_1           0.011602
2_2           0.003570
Name: proportion, dtype: float64

Sampled Sample_Type proportions:
Sample_Type
DIAGNOSTIC    0.725572
RESECTION     0.274428
Name: proportion, dtype: float64


In [57]:
expression_test.shape

(18460, 481)

In [58]:
test_metadata.shape

(481, 70)

# Signature Score

The signature scores are for PanCK segments only

In [25]:
signature_scores = pd.read_csv(os.path.join(file_path,"signature_scores.csv"), index_col=0)
signature_scores.head()

,EMT_Miow_Epithelial,EMT_Miow_Mesenchymal,EMT_Mak,EMT_Cheng,Pyroptosis_Ye,Ferroptosis_Ye,LipidMetabolism_Zheng,Hypoxia_Buffa,IPS_Charoentong_MHC,IPS_Charoentong_CP,...,ECM_Chakravarthy_down,HRDS_Lu,VEGF_Hu,LRRC15CAF_Dominguez,ICBResponse_Chen_responder,ICBResponse_Chen_nonresponder,BreastState_Wu_Basal,BreastState_Wu_Her2E,BreastState_Wu_LumA,BreastState_Wu_LumB
DSP-1001660008694-G-A07.dcc,-1.016896,0.646749,-1.311983,-1.238825,0.862967,0.531420,2.074673,1.105722,-1.728959,0.354099,...,-1.133261,1.435874,1.183403,-1.397861,-1.196211,0.910331,0.571354,-1.719320,-0.388388,-1.230769
DSP-1001660008694-G-A09.dcc,-0.984435,-0.379346,1.420660,1.254408,1.302024,0.244835,-2.194416,-1.978670,0.067006,0.022225,...,1.975692,-1.158890,-0.978927,-0.092230,0.385807,-2.612812,-0.972404,-0.812911,0.622242,1.261290
DSP-1001660008694-G-A11.dcc,-0.217836,-0.252731,0.011755,-0.018446,1.290781,-0.711057,0.185236,-0.681088,-0.783328,-0.767688,...,1.549937,-1.416203,-0.297675,-0.575398,0.257179,-1.695098,0.217885,-2.245142,-0.398401,-1.982081
DSP-1001660008694-G-B01.dcc,0.035143,-1.483572,0.167068,-0.293014,0.832096,0.153249,1.385377,-2.450660,-0.462663,0.388537,...,1.962419,-1.392234,-1.534590,-0.258684,0.308090,-2.790169,0.200933,-1.115613,-0.521340,-1.240963
DSP-1001660008694-G-B05.dcc,-0.411838,-1.407967,1.067610,0.319949,2.119825,-3.338973,-1.206909,-1.645790,-0.884179,0.239768,...,1.974837,-0.570433,-0.880439,-0.366735,0.294521,-2.434542,-0.323462,-1.549168,-0.504941,-1.194458


In [26]:
sample_metadata[sample_metadata['Segment']=='PanCK+'].shape

(2381, 70)

In [59]:
rois_panck_train = list(set(signature_scores.index).intersection(set(rois_train)))
sample_metadata.loc[rois_panck_train,'Segment'].value_counts()

Segment
PanCK+    2105
Name: count, dtype: int64

In [60]:
rois_panck_test = list(set(signature_scores.index).intersection(set(rois_test)))
sample_metadata.loc[rois_panck_test,'Segment'].value_counts()

Segment
PanCK+    190
Name: count, dtype: int64

## Define relevant sig

we are most interested in signatures that are likely to be visible in the H&E:

- EMT_Miow_Mesenchymal (	EMT strongly alters tumor architecture (spindle shape, invasion, stroma))
- EMT_Mak (Tumor phenotype shift visible in H&E)
- BreastState_Wu_Basal (Basal subtype has distinct H&E appearance (high-grade, mitotic, lymphocyte-rich))

possibly in 40x:
- MitoticIndex_Yang

In [34]:
signatures_to_drop = ['EMT_Miow_Epithelial', 'EMT_Mak', 'BreastState_Wu_Basal']
signatures_to_use = list(signature_scores.columns)

In [61]:
#sig_subset = signature_scores.loc[rois_panck,signatures_to_use]
#sig_subset = signature_scores.loc[rois_panck,:].drop(signatures_to_drop, axis=1)
sig_subset_train = signature_scores.loc[rois_panck_train,:]
sig_subset_test = signature_scores.loc[rois_panck_test,:]
sig_subset_train.head()

,EMT_Miow_Epithelial,EMT_Miow_Mesenchymal,EMT_Mak,EMT_Cheng,Pyroptosis_Ye,Ferroptosis_Ye,LipidMetabolism_Zheng,Hypoxia_Buffa,IPS_Charoentong_MHC,IPS_Charoentong_CP,...,ECM_Chakravarthy_down,HRDS_Lu,VEGF_Hu,LRRC15CAF_Dominguez,ICBResponse_Chen_responder,ICBResponse_Chen_nonresponder,BreastState_Wu_Basal,BreastState_Wu_Her2E,BreastState_Wu_LumA,BreastState_Wu_LumB
DSP-1001660013337-F-D05.dcc,-0.037075,-1.539543,0.533738,1.136892,0.598017,-0.933865,1.779060,-0.187995,1.217858,-1.471147,...,1.181262,-0.170901,-0.399890,0.323763,1.953786,-0.139758,-1.270735,-1.140917,-0.155928,-1.204885
DSP-1001660016637-F-C12.dcc,0.502778,0.128292,0.549963,0.164653,-0.032683,0.825163,0.697274,-0.563133,1.130154,-1.390781,...,0.647057,-1.897627,-0.089000,0.610577,0.508288,0.051693,1.236115,-0.116803,0.552479,0.819742
DSP-1001660014543-A-A03.dcc,1.294301,0.360311,-0.296095,-0.285914,-0.002103,-0.105039,0.038289,0.392740,0.958142,-0.932847,...,-0.590415,-0.689589,0.596285,0.116053,0.845386,0.667052,0.790009,0.466264,1.230926,0.650756
DSP-1001660020816-A-B01.dcc,-7.526882,3.478261,2.787850,0.298336,0.970953,0.844237,0.152605,-1.235054,-1.616317,0.227988,...,-0.257314,0.678343,-1.165095,-0.061205,-0.956777,0.191610,-1.166610,0.051384,-1.321715,-1.023150
DSP-1001660014543-A-C06.dcc,1.189661,0.152587,-0.073640,0.112480,0.069763,0.304268,1.509196,-0.002239,1.050229,-1.119218,...,-0.606801,-0.702521,-0.425428,0.141873,0.854333,0.799765,-0.759115,-1.373594,-1.395381,-0.744240


save the signature scores

In [62]:
sig_subset_train = sig_subset_train.reset_index().rename(columns={"index": "ROI_ID"})
sig_subset_test = sig_subset_test.reset_index().rename(columns={"index": "ROI_ID"})

In [63]:
sig_subset_train.to_csv(os.path.join(input_path,'truthlabels',f"panck_signatures_train.csv"), index=False)
sig_subset_train.to_csv(os.path.join(input_path,'truthlabels',f"panck_signatures_test.csv"), index=False)

# Gene Expression

<div class="alert alert-block alert-warning">
I should log1p normalise before saving
</div>

In [94]:
log_transformed_expr = expression_matrix.apply(np.log1p).T

In [97]:
rois_cd45_train = list(train_metadata[train_metadata['Segment']=='CD45+'].index)
rois_panck_train = list(train_metadata[train_metadata['Segment']=='PanCK+'].index)

In [98]:
rois_cd45_test = list(test_metadata[test_metadata['Segment']=='CD45+'].index)
rois_panck_test = list(test_metadata[test_metadata['Segment']=='PanCK+'].index)

## Split Segments

## Gene Panel

Focus on spatially informative genes such as GATA3, KRT5, CD68, or ESR1
Triple-Negative (TNBC)	EGFR, KRT5, KRT17, TP63
Proliferation	MKI67, TOP2A, BIRC5 (Survivin)


### Split PanCK (epithelial) from CD45 (immune)

CD45 gene panel:
CD3D, CD8A, CD4, FOXP3, CD68, CD163

PanCK gene panel:
KRT5, KRT14, TP63, MKI67, VIM, MMP9, ACTA2

In [79]:
'CD274' in list(expression_matrix.index)

True

CD274 = PD-L1

In [99]:
gene_panel_cd45 = ['CD274','CXCL9','IFNG','BCL2','CD3D', 'CD8A', 'CD4', 'FOXP3', 'CD68', 'CD163','IDO1', 'LAG3', 'STAT1', 'S100A9']
gene_panel_panck = ['BRCA1','BRCA2', 'TP53', 'PIK3CA','MYC','EGFR', 'AR', 'PTEN', 'RB1', 'CD274','BCL2','KRT5', 'KRT14', 'TP63', 'MKI67', 'VIM', 'MMP9','SOX10', 'ALDH1A1', 'CD44', 'ZEB1', 'SNAI1', 'CCNB1']

In [100]:
panel_df_panck = log_transformed_expr.loc[rois_panck_train,gene_panel_panck].reset_index().rename(columns={"index": "ROI_ID"})
panel_df_cd45 = log_transformed_expr.loc[rois_cd45_train,gene_panel_cd45].reset_index().rename(columns={"index": "ROI_ID"})

In [ ]:
#panel_df_panck = expression_matrix.T.loc[rois_panck,gene_panel_panck].reset_index().rename(columns={"index": "ROI_ID"})
#panel_df_cd45 = expression_matrix.T.loc[rois_cd45,gene_panel_cd45].reset_index().rename(columns={"index": "ROI_ID"})

In [101]:
panel_df_panck.to_csv(os.path.join(input_path,'truthlabels',f"gene_panel_panck_raw.csv"), index=False)
panel_df_cd45.to_csv(os.path.join(input_path,'truthlabels',f"gene_panel_cd45_raw.csv"), index=False)

TEST SET

In [ ]:
panel_df_panck = log_transformed_expr.loc[rois_panck_test,gene_panel_panck].reset_index().rename(columns={"index": "ROI_ID"})
panel_df_cd45 = log_transformed_expr.loc[rois_cd45_test,gene_panel_cd45].reset_index().rename(columns={"index": "ROI_ID"})
panel_df_panck.to_csv(os.path.join(input_path,'truthlabels',f"gene_panel_panck_raw_TEST.csv"), index=False)
panel_df_cd45.to_csv(os.path.join(input_path,'truthlabels',f"gene_panel_cd45_raw_TEST.csv"), index=False)

## HVGs

In [ ]:
sample_metadata.columns

In [ ]:
import scanpy as sc
import squidpy as sq
from anndata import AnnData

In [ ]:
final_expression_matrix = expression_matrix.T.astype(float).sort_index()
sample_metadata = sample_metadata.sort_index()

In [ ]:
log_transformed_expr = final_expression_matrix.apply(np.log1p)

In [ ]:
sample_metadata['ROICoordinateY']

In [ ]:
# Create AnnData object
adata = AnnData(final_expression_matrix) #expression matrix transposed

adata.obsm["spatial"] = sample_metadata[['ROICoordinateX', 'ROICoordinateY']].to_numpy()  #roi coords
adata.obs["LEAPID"] = sample_metadata['LEAP_ID']
adata.obs["segment"] = sample_metadata['Segment']

In [ ]:
adata.X = np.nan_to_num(adata.X, nan=0.0, posinf=0.0, neginf=0.0)

In [ ]:
from collections import defaultdict

# Dictionary to store HVG results per segment
hvg_results_by_segment = {}

# Loop over each segment type
for segment in ['CD45+','PanCK+']:
    print(f"Processing segment: {segment}")
    
    # Subset AnnData
    adata_sub = adata[adata.obs["segment"] == segment].copy()
    
    # Filter out genes with all-zero expression (optional but good)
    sc.pp.filter_genes(adata_sub, min_cells=1)
    
    # Identify highly variable genes
    sc.pp.highly_variable_genes(
        adata_sub,
        flavor='cell_ranger',
        n_top_genes=100
    )
    
    # Store DataFrame of HVGs
    hvg_df = adata_sub.var[adata_sub.var["highly_variable"]].copy()
    hvg_results_by_segment[segment] = hvg_df

    print(f"Found {hvg_df.shape[0]} highly variable genes for {segment}")


Get top 50 HVGs per segment

In [ ]:
HVGs_cd45 = list(hvg_results_by_segment['CD45+'].sort_values(by='dispersions_norm', ascending=False)[:50].index)
HVGs_cd45

In [ ]:
HVGs_panck = list(hvg_results_by_segment['PanCK+'].sort_values(by='dispersions_norm', ascending=False)[:50].index)
HVGs_panck

In [ ]:
from collections import defaultdict

top_n = 100
segment_to_top_genes = {
    segment: set(hvg_df.sort_values("dispersions_norm", ascending=False).index[:top_n])
    for segment, hvg_df in hvg_results_by_segment.items()
}

# Count how many segments each gene appears in
from collections import Counter

gene_counter = Counter()
for genes in segment_to_top_genes.values():
    gene_counter.update(genes)

# Show most common HVGs
print("Most frequently variable genes across segments:")
for gene, count in gene_counter.most_common(10):
    print(f"{gene}: {count} segments")

In [ ]:
hvgs_df_panck = log_transformed_expr.loc[rois_panck,HVGs_panck].reset_index().rename(columns={"index": "ROI_ID"})
hvgs_df_cd45 = log_transformed_expr.loc[rois_cd45,HVGs_cd45].reset_index().rename(columns={"index": "ROI_ID"})

In [ ]:

#hvgs_df_panck = expression_matrix.T.loc[rois_panck,HVGs_panck].reset_index().rename(columns={"index": "ROI_ID"})
#hvgs_df_cd45 = expression_matrix.T.loc[rois_cd45,HVGs_cd45].reset_index().rename(columns={"index": "ROI_ID"})

In [ ]:
hvgs_df_panck.to_csv(os.path.join(input_path,'truthlabels',f"hvgs_panck_raw.csv"), index=False)
hvgs_df_cd45.to_csv(os.path.join(input_path,'truthlabels',f"hvgs_cd45_raw.csv"), index=False)

## SVGs

In [ ]:
results = {}

for leapid in adata.obs["LEAPID"].unique():
    for segment in ['CD45+','PanCK+']:
        subset = adata[
            (adata.obs["LEAPID"] == leapid) &
            (adata.obs["segment"] == segment)
        ].copy()
        

        # Compute spatial graph and spatial autocorrelation
        n_obs = subset.n_obs

        if n_obs > 1:
            #print(f"{leapid} {segment} {n_obs}")
            n_neighbors = min(5, n_obs - 1)


            # Build spatial graph and compute Moran’s I
            sq.gr.spatial_neighbors(subset, coord_type="generic", n_neighs=n_neighbors)
            sq.gr.spatial_autocorr(subset, mode="moran")

            # Extract top genes
            moranI = subset.uns['moranI']
            top_genes = moranI.sort_values("I", ascending=False).head(10)

            results[(leapid, segment)] = top_genes
            #print(f"Top genes for {leapid}-{segment}: {top_genes.index.tolist()}")

In [ ]:
from collections import defaultdict, Counter

# Dictionary: segment → Counter of gene frequencies
gene_counts_by_segment = defaultdict(Counter)

top_n = 20  # Adjust based on how many top genes you consider

for (leapid, segment), df in results.items():
    top_genes = df.head(top_n).index.tolist()
    gene_counts_by_segment[segment].update(top_genes)

Get top 50 SVGs per segment

In [ ]:
SVGs_cd45 = [gene for segment, counter in gene_counts_by_segment.items() for gene,_ in counter.most_common(50) if segment=='CD45+']
SVGs_cd45

In [ ]:
SVGs_panck = [gene for segment, counter in gene_counts_by_segment.items() for gene,_ in counter.most_common(50) if segment=='PanCK+']
SVGs_panck

In [ ]:
svgs_df_panck = log_transformed_expr.loc[rois_panck,SVGs_panck].reset_index().rename(columns={"index": "ROI_ID"})
svgs_df_cd45 = log_transformed_expr.loc[rois_cd45,SVGs_cd45].reset_index().rename(columns={"index": "ROI_ID"})

In [ ]:

#svgs_df_panck = expression_matrix.T.loc[rois_panck,SVGs_panck].reset_index().rename(columns={"index": "ROI_ID"})
#svgs_df_cd45 = expression_matrix.T.loc[rois_cd45,SVGs_cd45].reset_index().rename(columns={"index": "ROI_ID"})

In [ ]:
svgs_df_panck.to_csv(os.path.join(input_path,'truthlabels',f"svgs_panck_raw.csv"), index=False)
svgs_df_cd45.to_csv(os.path.join(input_path,'truthlabels',f"svgs_cd45_raw.csv"), index=False)

## Single Gene

Select whether CD45 or PanCK

In [ ]:
segment_expression_matrix = expression_matrix[rois_cd45]


Get expression data for just ROIs from the specified segment

In [ ]:
# Select a specific gene
gene = "IGHG4"  # Replace with the gene of interest

print(gene in expression_matrix.index)
print(gene in segment_expression_matrix.index)


### Visualize gene distribution

#### Whole Distribution of gene

In [ ]:
expression_data = segment_expression_matrix.loc[gene,:] 
expression_df = pd.DataFrame({
    "ROI_ID": expression_data.index ,
    f"{gene}": expression_data.values,
})
gene_expr = expression_df[gene]

In [ ]:

single_violin(gene_expr,gene)

#### Histplot

In [ ]:
sns.histplot(gene_expr, kde=True, bins=50)
plt.title(f'Distribution of {gene} expression')
plt.xlabel('Expression Value')
plt.ylabel('Frequency')
plt.show()

In [ ]:
summary = pd.Series(gene_expr).describe()
print(summary)

#### Boxplot for outliers

In [ ]:
sns.boxplot(x=gene_expr)
plt.title(f'Boxplot of {gene} Expression')
plt.show()

#### Skewness & Kurtosis
Skewness > 1 or < -1 = highly skewed

Kurtosis > 3 = heavy tails (long-tailed)


In [ ]:
from scipy.stats import skew, kurtosis

print(f"Skewness: {skew(gene_expr)}")
print(f"Kurtosis: {kurtosis(gene_expr)}")

#### QQ Plot
Strong upward bend = right-skewed / long tail

In [ ]:
import scipy.stats as stats

stats.probplot(gene_expr, dist="norm", plot=plt)
plt.title(f'Q-Q Plot of {gene} Expression')
plt.show()

consider log transforming the expression 
y_train_log = np.log1p(y_train) 

### Save Expression csv

Initialise outcomes to all zeros

save to csv

In [ ]:
expression_df.to_csv(os.path.join(input_path,'truthlabels',f"{gene}_{segment}_expression.csv"), index=False)

# END
